In [1]:
!pip install transformers==4.29
!pip install einops
!pip install triton==2.0.0.dev20221202

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 6.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 8.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 9.3 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00
Defaulting to use

In [3]:
import pandas as pd

# Read CSV file using pandas
file_path = '../protein_embeddings/data/rbp.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,Protein ID,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,...,Genbank Division,Isolation Host (beware inconsistent and nonsense values),Year-Month,Host Superkingdom,Host Phylum,Host Class,Host Order,Host Family,Protein Sequence,Nucleotide Sequence
0,BAF36105.1,AB231700,Microcystis virus Ma-LMM01,Microcystis virus Ma-LMM01 Fukuivirus Caudovir...,162109,False,45.953,DNA,2021-07-14,189,...,PHG,Unspecified,2021-07,bacteria,cyanobacteria,NaN,chroococcales,microcystaceae,MHQNISKENRGNYNNGIRPRIFMITTIDFRDIQAACIKQLDDMSKD...,GTGCATCAAAATATTTCAAAGGAGAATCGTGGAAACTATAACAACG...
1,BAF36110.1,AB231700,Microcystis virus Ma-LMM01,Microcystis virus Ma-LMM01 Fukuivirus Caudovir...,162109,False,45.953,DNA,2021-07-14,189,...,PHG,Unspecified,2021-07,bacteria,cyanobacteria,NaN,chroococcales,microcystaceae,MRIFYIHHPFLATHRYLLSNAYSTPYTDSITKLTTSYSSMPIILSV...,GTGAGGATTTTTTATATCCACCATCCATTCCTCGCTACTCACCGAT...
2,BAF36131.1,AB231700,Microcystis virus Ma-LMM01,Microcystis virus Ma-LMM01 Fukuivirus Caudovir...,162109,False,45.953,DNA,2021-07-14,189,...,PHG,Unspecified,2021-07,bacteria,cyanobacteria,NaN,chroococcales,microcystaceae,MLTDVDIQALIDASISGLSGEMPIVANIAARNALSLTKNTQVLVLD...,TTGCTGACAGATGTCGATATTCAGGCATTAATTGATGCCTCAATTT...
3,BAF36132.1,AB231700,Microcystis virus Ma-LMM01,Microcystis virus Ma-LMM01 Fukuivirus Caudovir...,162109,False,45.953,DNA,2021-07-14,189,...,PHG,Unspecified,2021-07,bacteria,cyanobacteria,NaN,chroococcales,microcystaceae,MFGVFIVRREGGYIGTQPNWDAANRPGNWDILDVYNRQRRNLWIQS...,TTGTTCGGAGTTTTTATCGTGAGGCGTGAAGGCGGCTATATCGGAA...
4,BAF36193.1,AB231700,Microcystis virus Ma-LMM01,Microcystis virus Ma-LMM01 Fukuivirus Caudovir...,162109,False,45.953,DNA,2021-07-14,189,...,PHG,Unspecified,2021-07,bacteria,cyanobacteria,NaN,chroococcales,microcystaceae,MVNYRYRLSRLLIPGGIPDPEIGEVELFLASDRQGYINNIDLPPDP...,TTGGTTAATTATCGTTATAGATTATCACGACTACTAATCCCGGGGG...


In [5]:
DNA_seq = df["Nucleotide Sequence"] 
len(DNA_seq) 

24752

In [7]:
import torch
import contextlib
import io
from transformers import AutoTokenizer, AutoModel

# download the model and tokenizer
with contextlib.redirect_stdout(io.StringIO()): 
    tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True) 
    model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to b

In [9]:
DNA_embeddings = []
model = model#.to("cuda")

for i in range(len(DNA_seq)):

  dna = DNA_seq[i]

  inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]

  inputs = inputs#.to("cuda")

  with torch.no_grad():
    hidden_states = model(inputs)[0] # [1, sequence_length, 768]

  # embedding with mean pooling
  embedding_mean = torch.mean(hidden_states[0], dim=0)
  DNA_embeddings.append(embedding_mean) # expect to be 768

  inputs#.detach().cpu()
  hidden_states#.detach().cpu()



DNA_embeddings_tensor = torch.stack(DNA_embeddings)
torch.save(DNA_embeddings_tensor, "./embeddings_DB2.pt")
print(DNA_embeddings_tensor)

/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:432: UserWarning: Increasing alibi size from 512 to 591
  warnings.warn(
/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:432: UserWarning: Increasing alibi size from 591 to 867
  warnings.warn(
/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:432: UserWarning: Increasing alibi size from 867 to 879
  warnings.warn(
/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:432: UserWarning: Increasing alibi size from 879 to 989
  warnings.warn(
/Users/sumanth/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb8

KeyboardInterrupt: 